In [390]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Reading

In [391]:
import pandas as pd
pd.set_option('display.max_colwidth',100)

In [392]:
Data=pd.read_excel('drive/My Drive/Ankit/2020 Dataset/new data/hasoc_2020_hi_train.xlsx',sep='\t')
#Data=pd.read_excel('drive/My Drive/Summer Internship 2020 July/My Test File/Ankit/2020 Dataset/new data/hasoc_2020_hi_train.xlsx',sep='\t')
Data.head()


,tweet_id,text,task1,task2,ID
0,1127755185944711170,"1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो\n17000 सिखो, 5000 भोपाली, 3000 तमिलों का कत्लेआम कर...",HOF,HATE,hasoc_2020_hi_2250
1,1123578938406592513,"RT @Vishesh4: @jawaharyadavbjp जवाहर यादव, अगर हिम्मत है तो पूरा वीडियो डालो फिर तुम्हारे झूठ सा...",NOT,NONE,hasoc_2020_hi_381
2,1127750010156642304,RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही कर सकते लेकिन बुरखा पहन के कर सकते हैं @yadavakhil...,HOF,HATE,hasoc_2020_hi_1510
3,1127660759553597441,Yey nina khothani labafazi benu phambili Finish and klaar 🍑🌊,HOF,PRFN,hasoc_2020_hi_1010
4,1123487255136489472,RT @Rajeshbhanjan2: जब भी कोई सिकुलर कोंग्रेसी गुलाम चाटुकार चरणचाट चमचा 'गाय' और 'बछड़े' को लेक...,HOF,HATE,hasoc_2020_hi_331


# Data Preprocessing
REmoving emoji

In [393]:
#rem0ving emoji

import re

def deEmojify(txt):
    regrex_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',txt)

Data['text']=Data['text'].apply(lambda x:deEmojify(x))
Data[['text','task2']].head()

,text,task2
0,"1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो\n17000 सिखो, 5000 भोपाली, 3000 तमिलों का कत्लेआम कर...",HATE
1,"RT @Vishesh4: @jawaharyadavbjp जवाहर यादव, अगर हिम्मत है तो पूरा वीडियो डालो फिर तुम्हारे झूठ सा...",NONE
2,RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही कर सकते लेकिन बुरखा पहन के कर सकते हैं @yadavakhil...,HATE
3,Yey nina khothani labafazi benu phambili Finish and klaar,PRFN
4,RT @Rajeshbhanjan2: जब भी कोई सिकुलर कोंग्रेसी गुलाम चाटुकार चरणचाट चमचा 'गाय' और 'बछड़े' को लेक...,HATE


Removing URL

In [394]:
# remove URL
def remove_URL(txt):
    url= re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"",txt)

Data['text']=Data['text'].apply(lambda x:remove_URL(x))
Data[['text']].head()

Data['text']=Data['text'].str.replace(r'\d+','')
Data[['text','task2']].head()

,text,task2
0,"आदमीं को मारने पर गोडसे आतंकी हो सके है तो\n सिखो, भोपाली, तमिलों का कत्लेआम करवाने वाला भारत...",HATE
1,"RT @Vishesh: @jawaharyadavbjp जवाहर यादव, अगर हिम्मत है तो पूरा वीडियो डालो फिर तुम्हारे झूठ साम...",NONE
2,RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही कर सकते लेकिन बुरखा पहन के कर सकते हैं @yadavakhil...,HATE
3,Yey nina khothani labafazi benu phambili Finish and klaar,PRFN
4,RT @Rajeshbhanjan: जब भी कोई सिकुलर कोंग्रेसी गुलाम चाटुकार चरणचाट चमचा 'गाय' और 'बछड़े' को लेकर...,HATE


# Tokenization, Encoding, padding after preprocessing

In [395]:
import keras
from keras.preprocessing.text import Tokenizer
tok1 = Tokenizer(char_level=False, filters ='!"$%&@()*+,-./:;”“<=>?[\\]^_`{|}~\t\n', lower = True)

In [396]:
tok1.fit_on_texts(Data['text'])
words=len(tok1.word_counts)
words

13545

In [397]:
word_list=[tok1.word_index]
#word_list

In [398]:
encoded=tok1.texts_to_sequences(Data['text'])


In [399]:
##         Padding encoded sequence of words
from keras.preprocessing import sequence
max_length=20
padd = sequence.pad_sequences(encoded, maxlen=max_length, padding='post')
#padd[0:12]

# Label Encoding

In [400]:
import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing
labelEncode=preprocessing.LabelEncoder()
labelEncode.fit(Data['task2'])
print (labelEncode.classes_)
train_labelEncode=labelEncode.transform(Data['task2'])
train_label=to_categorical(np.asarray(train_labelEncode))

['HATE' 'NONE' 'OFFN' 'PRFN']


# Importing libraries for Model creation

In [401]:
# importing libraries for creating model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Embedding,MaxPool1D
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D

#CNN model

In [513]:
model = Sequential()
model.add(Embedding(input_dim = words + 1, output_dim = 100,input_length=max_length))
model.add(Conv1D(filters =512,kernel_size=2, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(4, activation = 'sigmoid'))
model.summary()

Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_33 (Embedding)     (None, 20, 100)           1354600   
_________________________________________________________________
conv1d_41 (Conv1D)           (None, 19, 512)           102912    
_________________________________________________________________
max_pooling1d_41 (MaxPooling (None, 6, 512)            0         
_________________________________________________________________
flatten_29 (Flatten)         (None, 3072)              0         
_________________________________________________________________
dense_73 (Dense)             (None, 128)               393344    
_________________________________________________________________
dense_74 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_75 (Dense)             (None, 4)               

# Model Compilation

In [514]:
## compile

model.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['acc'])

# Model Training

In [ ]:
model.fit(padd,train_label,epochs=50,verbose=1,batch_size= 16)

Epoch 1/50
186/186 [==============================] - 1s 7ms/step - loss: 0.3846 - acc: 0.7111
Epoch 2/50
186/186 [==============================] - 1s 7ms/step - loss: 0.3255 - acc: 0.7202
Epoch 3/50
186/186 [==============================] - 1s 7ms/step - loss: 0.2152 - acc: 0.8110
Epoch 4/50
186/186 [==============================] - 1s 7ms/step - loss: 0.1298 - acc: 0.8829
Epoch 5/50
186/186 [==============================] - 1s 8ms/step - loss: 0.0677 - acc: 0.9507
Epoch 6/50
186/186 [==============================] - 1s 7ms/step - loss: 0.0258 - acc: 0.9885
Epoch 7/50
186/186 [==============================] - 1s 7ms/step - loss: 0.0126 - acc: 0.9946
Epoch 8/50
186/186 [==============================] - 1s 7ms/step - loss: 0.0107 - acc: 0.9966
Epoch 9/50
186/186 [==============================] - 1s 8ms/step - loss: 0.0072 - acc: 0.9976
Epoch 10/50
186/186 [==============================] - 1s 7ms/step - loss: 0.0088 - acc: 0.9963
Epoch 11/50
186/186 [============================

#Test data reading

In [ ]:
Test =pd.read_csv('drive/My Drive/Ankit/2020 Dataset/test data/hindi_test_1509.csv')
#Test =pd.read_csv('drive/My Drive/Summer Internship 2020 July/My Test File/Ankit/2020 Dataset/test data/hindi_test_1509.csv')
Test.dropna(inplace = True)
Test.head()

#Test data preprocessing

In [ ]:
## doing preprocessing as above

Test['text']=Test['text'].apply(lambda x:deEmojify(x))
Test['text']=Test['text'].apply(lambda x: remove_URL(x))
Test['text']=Test['text'].str.replace(r'\d+','')
t= Test['text']

# Test data encoding, padding

In [ ]:
encoded1 =tok1.texts_to_sequences(t)
#print(t[0])
#encoded1[0]

In [ ]:
padded = sequence.pad_sequences(encoded1, maxlen=max_length, padding='post')
padded[0]

#Test label encoding

In [ ]:
test_labelEncode=labelEncode.transform(Test['task2'])
test_label=to_categorical(np.asarray(test_labelEncode))


# Test data Model prediction

In [ ]:
test_predictions=model.predict(padded)
test_predictions2=np.argmax(test_predictions,axis=1)

# Test data Classification report

In [ ]:
#Accuracy 
from sklearn.metrics import classification_report
print(classification_report(test_labelEncode,test_predictions2))

# Adding predicted lable as a colum to dataFrame 'Test'

In [424]:
Test['Predictionlabel']=labelEncode.inverse_transform(test_predictions2)
Test.head()

,tweet_id,text,task1,task2,ID,Predictionlabel
0,1127781647783301120,@Kinjal_Dubey_ इसके लिए तैयारी हो रखी है\nआपको जान देने की जरूरत नहीं है।\nरक्षा मंत्री एक महिला...,HOF,OFFN,hasoc_2020_hi_1938,NONE
1,1127492853171535872,"RT @pratimamishra: दिल्ली के मुख्यमंत्री @ArvindKejriwal ने अपने पूरे परिवार के साथ वोट डाला, कह...",NOT,NONE,hasoc_2020_hi_1201,NONE
2,1123638375896903681,"RT @nishantdabre: आज एक तरफ खुशी है, तो एक तरफ दुख\nएक तरफ मसूद अजहर को पूरे विश्व के आतंकी के र...",NOT,NONE,hasoc_2020_hi_1628,OFFN
3,1127496460310683648,@BJPIndia @HardeepSPuri जिस कुनबे के लोग :-\nवीं पास डिग्री छुपाने वाली को शिक्षा मंत्री बनाते ह...,NOT,NONE,hasoc_2020_hi_3257,NONE
4,1127667277535399937,@manakgupta बादल....,NOT,NONE,hasoc_2020_hi_4133,NONE


# Generating CSV file

In [425]:
HindiTask2File=Test[['tweet_id','Predictionlabel','ID']]
HindiTask2File.head()
HindiTask2File.to_csv('submission_HI_B.csv',index=False)